In [8]:
!C:/Users/SSAFY/AppData/Local/Programs/Python/Python310/python.exe -m pip install pyrender PyOpenGL PyOpenGL_accelerate



     ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
     ---------------------------------------- 2.2/2.2 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/984.0 kB ? eta -:--:--
   ---------------------------------------- 984.0/984.0 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/814.6 kB ? eta -:--:--
   ---------------------------------------- 814.6/814.6 kB 7.1 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1749286 sha256=483bbf272fdecc8eb7d4cd6156872964d3500ce6d92f9eaa7de518e625a0d089
  Stored in directory: c:\users\ssafy\appdata\local\pip\cache\wheels\a1\3c\d2\1f9533f908d86176637521e533c6cdb2d4e48b59003b5c3f19
Successfully bui

  DEPRECATION: Building 'PyOpenGL' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'PyOpenGL'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [13]:
import trimesh

model = trimesh.load("dd.glb")
print(model.graph.nodes_geometry)


['Hair', 'Body_56b712', 'Body_3c7e84', 'Body_34cf5e', 'Body_e4f464', 'Face_74fb02', 'Face_d8ab8e', 'Face_896755', 'Face_0aa89a', 'Face_fb0d0b', 'Face_d9f266', 'Face_3c0869']


In [12]:
import cv2
import numpy as np
import mediapipe as mp
import trimesh
import pyrender

# Mediapipe 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
cap = cv2.VideoCapture(0)

# pyrender scene 초기화
scene = pyrender.Scene()

# 카메라 추가
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
cam_node = scene.add(camera, pose=np.array([
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, -0.3],  # 카메라 위치 (Y축으로 살짝 뒤로)
    [0.0, 0.0, 1.0, 1.0],   # 카메라 Z축 거리
    [0.0, 0.0, 0.0, 1.0]
]))

# GLB 모델 불러오기
model = trimesh.load("dd.glb")
nodes = []
for name, geom in model.geometry.items():
    mesh = pyrender.Mesh.from_trimesh(geom)
    node = scene.add(mesh)
    nodes.append(node)

# 렌더러 설정
renderer = pyrender.OffscreenRenderer(viewport_width=640, viewport_height=480)

# 카메라 내부 파라미터
camera_matrix = np.array([
    [640, 0, 320],
    [0, 640, 240],
    [0,   0,   1]
], dtype=np.float64)
dist_coeffs = np.zeros((4, 1))

# 얼굴 기준점 (solvePnP용)
face_3d = np.array([
    [0, 0, 0],             # nose
    [0, -63.6, -12.5],     # chin
    [-43.3, 32.7, -26.0],  # left eye
    [43.3, 32.7, -26.0],   # right eye
    [-28.9, -28.9, -24.1], # left mouth
    [28.9, -28.9, -24.1]   # right mouth
], dtype=np.float64)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if result.multi_face_landmarks:
        lmk = result.multi_face_landmarks[0].landmark

        face_2d = np.array([
            [lmk[1].x * 640,   lmk[1].y * 480],
            [lmk[152].x * 640, lmk[152].y * 480],
            [lmk[263].x * 640, lmk[263].y * 480],
            [lmk[33].x * 640,  lmk[33].y * 480],
            [lmk[287].x * 640, lmk[287].y * 480],
            [lmk[57].x * 640,  lmk[57].y * 480],
        ], dtype=np.float64)

        success, rvec, tvec = cv2.solvePnP(face_3d, face_2d, camera_matrix, dist_coeffs)
        rot_mat, _ = cv2.Rodrigues(rvec)

        pose = np.eye(4)
        pose[:3, :3] = rot_mat
        pose[:3, 3] = tvec.flatten() * 0.01 + np.array([0, -0.2, 0])

        # 모든 파츠에 pose 적용
        for node in nodes:
            scene.set_pose(node, pose)

        color, _ = renderer.render(scene)
        cv2.imshow("Face Track", cv2.cvtColor(color, cv2.COLOR_RGB2BGR))

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
